# import

In [ ]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

# データの読み込み

In [ ]:
# MNIST データのロード
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# fashion　MNISTを使いたいときはこちら
#(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() 

# データの前処理

In [ ]:
# 0 - 255の画素データを 0.0 - 1.0に変換
x_train = x_train / 255.0
x_test = x_test / 255.0

# 2次元画素データを1次元に変換
x_train = x_train.reshape(-1, 784) 
x_test = x_test.reshape(-1, 784)

# one-hotベクトル化
y_train = keras.utils.to_categorical(y_train, 10) 
y_test = keras.utils.to_categorical(y_test, 10)

# モデルの構築

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation=tf.nn.relu, input_shape=(784,)),
    Dense(10, activation=tf.nn.softmax)
])

# コンパイル

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.5),
              loss='mean_squared_error',
              metrics=['accuracy'])

# モデル確認

In [ ]:
model.summary()

# モデルの可視化

In [ ]:
keras.utils.plot_model(model)

# モデルの保存

In [ ]:
model_json_str = model.to_json()
open('model.json', 'w').write(model_json_str)
model.save_weights('weights.h5');

# Callbackの定義

In [ ]:
tb_cb = keras.callbacks.TensorBoard(
    log_dir="logs/",   # ログの出力先
    histogram_freq=1,  # ヒストグラムの更新頻度
    write_graph=True,  # 計算グラフを出力するか？
    write_images=True  # パラメータを可視化して表示するか？
)

In [ ]:
if tf.__version__[0]=='1':
    sv_cb = keras.callbacks.ModelCheckpoint(
        filepath = 'save/ckpt/model',  # セーブデータの出力先
        monitor='val_loss',            # save_best_onlyがTrueの時に参照する指標
        verbose=0,                     # ログを出力するか？
        save_best_only=False,          # monitorで指定した指標が良い時だけ上書きする
        save_weights_only=True,        # ウェイトのみを出力するか？（グラフは出力しない）
        mode='auto',                   # save_best_onlyがTrueの時の判定方法
        save_freq=1                    # セーブの頻度
    )
else:
    sv_cb = keras.callbacks.ModelCheckpoint(
        filepath = 'save/ckpt/model',
        monitor='val_loss',
        verbose=0,
        save_best_only=False,
        save_weights_only=True,
        mode='auto',
        period=1   # tf2ではsave_freqではなくperiod
    )

# 訓練

In [ ]:
EPOCHS = 15
model.fit(x_train, y_train, epochs=EPOCHS, validation_split=0.2, callbacks=[tb_cb, sv_cb])

# 評価

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
print(test_loss)
print(test_acc)

# 推論

In [ ]:
model.predict(x_test[:10])

# 訓練済みモデルの保存

### savedmodel

In [ ]:
import os

if tf.__version__[0]=='1':  # TF1のみ

    sess = keras.backend.get_session()
    tf.saved_model.simple_save(
        sess,
        os.path.join("save", "savedmodel/1"),
                     inputs={'inputs': model.input}, 
                     outputs={t.name: t for t in model.outputs}

    )

In [ ]:
keras.experimental.export_saved_model(model, "./save/experimental")

### hdf5

In [ ]:
model.save("save/model.h5")